In [19]:
!pip install --quiet qiskit-optimization\[cplex\] qiskit

In [25]:
ABU_DHABI = (0,3)
DUBAI = (5,0)


# ship = (capability, location)
ships = {
    "ship1": (5, ABU_DHABI),
    "ship2": (10, DUBAI)
}

# oilspill = (size, coordinates)
oilspill = (20, (3, 3))


def create_oil_qubo(ships, oilspill):
    mod = QuadraticProgram("oil spill")
    
    # create a binary variable for each ship indicating whether we should send it or not
    for ship in ships.keys():
        mod.binary_var(name=ship)
    
    # add the distance between each ship and the oilspill
    kilometers = {}
    for ship, data in ships.items():
        kilometers[ship] = 2.5  # TODO: actually calculate the distance
    mod.minimize(linear=kilometers)
                      
    # TODO: add capabilities and oil spill size as linear constraint
    return mod
    

qubo = create_oil_qubo(ships, oilspill)
print(qubo.export_as_lp_string())



\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: oil spill

Minimize
 obj: 2.500000000000 ship1 + 2.500000000000 ship2
Subject To

Bounds
 0 <= ship1 <= 1
 0 <= ship2 <= 1

Binaries
 ship1 ship2
End



In [16]:
from qiskit_optimization import QuadraticProgram

mod = QuadraticProgram("oil spill")


mod.binary_var(name="x")
mod.integer_var(name="y", lowerbound=-1, upperbound=5)
# mod.binary_var(name="z")

mod.minimize(linear={"x": 1, "y": 2, "y": 3}, quadratic={("x", "y"): 2})

mod.linear_constraint(linear={"x": 1, "y": 2}, sense=">=", rhs=3, name="lin_eq")

print(mod.export_as_lp_string())

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: oil spill

Minimize
 obj: x + 3 y + [ 4 x*y ]/2
Subject To
 lin_eq: x + 2 y >= 3

Bounds
 0 <= x <= 1
 -1 <= y <= 5

Binaries
 x

Generals
 y
End



In [35]:
from qiskit_optimization.algorithms import MinimumEigenOptimizer, CplexOptimizer

CplexOptimizer().solve(mod)

optimal function value: 6.0
optimal value: [0. 2.]
status: SUCCESS

In [ ]:
from qiskit import BasicAer, Aer
from qiskit.algorithms import QAOA
from qiskit_optimization.algorithms import MinimumEigenOptimizer
from qiskit.algorithms.optimizers import COBYLA, SLSQP, ADAM



backend = Aer.get_backend('statevector_simulator')
qaoa = QAOA(optimizer = ADAM(), quantum_instance = backend, reps=1)
eigen_optimizer = MinimumEigenOptimizer(min_eigen_solver = qaoa)
result = eigen_optimizer.solve(mod)
result